In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import pandas as pd
from PIL import Image
from torchsummary import summary
from torch.cuda.amp import GradScaler, autocast

In [2]:
allocated_memory = torch.cuda.memory_allocated()
print("Currently allocated GPU memory:", allocated_memory / 1024**2, "MB")

# Get peak memory usage
peak_memory = torch.cuda.max_memory_allocated()
print("Peak GPU memory usage:", peak_memory / 1024**2, "MB")

Currently allocated GPU memory: 0.0 MB
Peak GPU memory usage: 0.0 MB


In [3]:
num_gpus = torch.cuda.device_count()

# Get information about each GPU
for i in range(num_gpus):
    properties = torch.cuda.get_device_properties(i)
    print("GPU", i+1, "Total Memory:", properties.total_memory / (1024**2), "MB")

GPU 1 Total Memory: 6140.5 MB


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_image_path = self.dataframe.iloc[idx, 0]
        final_image_path = self.dataframe.iloc[idx, 1]
        input_image = Image.open(input_image_path)
        final_image = Image.open(final_image_path)
        if self.transform:
            input_image = self.transform(input_image)
            final_image = self.transform(final_image)
        return input_image, final_image

# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 448)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Dataset
dataframe = pd.read_csv("paths.csv")
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
# dataset = CustomDataset(dataframe, transform=transform)

# # DataLoader
# batch_size = 8
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) 


In [8]:
import torch
import torch.nn as nn

class DeblurModel(nn.Module):
    def __init__(self):
        super(DeblurModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeblurModel().to(device)
summary(model, (3, 256, 448))  # Print summary of the model
input_tensor = torch.randn(1, 3, 256, 448).to(device)

  
output_tensor = model(input_tensor)
print("Output tensor shape:", output_tensor.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 448]           1,792
       BatchNorm2d-2         [-1, 64, 256, 448]             128
              ReLU-3         [-1, 64, 256, 448]               0
            Conv2d-4         [-1, 64, 256, 448]          36,928
       BatchNorm2d-5         [-1, 64, 256, 448]             128
              ReLU-6         [-1, 64, 256, 448]               0
            Conv2d-7        [-1, 128, 256, 448]          73,856
       BatchNorm2d-8        [-1, 128, 256, 448]             256
              ReLU-9        [-1, 128, 256, 448]               0
           Conv2d-10        [-1, 128, 256, 448]         147,584
      BatchNorm2d-11        [-1, 128, 256, 448]             256
             ReLU-12        [-1, 128, 256, 448]               0
           Conv2d-13        [-1, 256, 256, 448]         295,168
      BatchNorm2d-14        [-1, 256, 2

In [9]:
allocated_memory = torch.cuda.memory_allocated()
print("Currently allocated GPU memory:", allocated_memory / 1024**2, "MB")

# Get peak memory usage
peak_memory = torch.cuda.max_memory_allocated()
print("Peak GPU memory usage:", peak_memory / 1024**2, "MB")

Currently allocated GPU memory: 2045.68310546875 MB
Peak GPU memory usage: 6574.123046875 MB


In [17]:
# state_dict = torch.load('trained_model_sid.pth')
# model.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
# dataframe=dataframe.iloc[:2000]

from torch.utils.data import Subset, random_split

dataset = CustomDataset(dataframe, transform=transform) 
# train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

# Split dataset into train and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create train and validation loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


In [11]:
scaler = GradScaler()

In [ ]:
num_epochs = 1

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

for epoch in range(num_epochs):
    total_train_loss = 0
    total_val_loss = 0
    
    # Training
    model.train()
    for i, (input_images, final_images) in enumerate(train_loader):
        input_images = input_images.to(device)
        final_images = final_images.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(input_images)
            train_loss = criterion(outputs, final_images)
        scaler.scale(train_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_train_loss += train_loss.item()

        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, len(train_loader), train_loss.item()))
    
    # Validation
    model.eval()
    with torch.no_grad():
        for i, (input_images, final_images) in enumerate(val_loader):
            input_images = input_images.to(device)
            final_images = final_images.to(device)

            outputs = model(input_images)
            val_loss = criterion(outputs, final_images)
            total_val_loss += val_loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)
    print('Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch + 1, num_epochs, avg_train_loss, avg_val_loss))

In [14]:
torch.save(model.state_dict(), 'trained_model_sid_2.pth')
print("Model saved successfully.")

Model saved successfully.
